### load library

In [3]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import os,sys, gc, keras

dirpath = os.getcwd().split("code")[0]+"code/"
sys.path.append(dirpath)
from pre_process import clean_LC,generate_representation

folder = "../../KOI_Data/"
#folder_lc = "/work/work_teamEXOPLANET/KOI_LC/"
folder_lc = "/media/fmena/KOI_LC/"

### load data

In [ ]:
time_kepler = np.load(folder_lc+"npy/KOI_LC_time.npy")
process_lc = np.load(folder_lc+'/cleaned/LC_kepler_processed.npy')
N, T = time_kepler.shape

plt.plot(time_kepler[0],process_lc[0])
plt.show()
print((N,T))

In [6]:
#borrar nans... arreglo variable
coupled_lc = []
delta_time = []
lens_lc = []
for i in range(N):
    mask_nan = np.isnan(process_lc[i])
    coupled_lc.append(process_lc[i][~mask_nan])
    
    time = time_kepler[i][~mask_nan]
    # calculate delta time --> this could be done after padding is done..
    delta_time.append(np.hstack([[0],np.diff(time)]))
    lens_lc.append(np.sum(~mask_nan))
    
coupled_lc = np.asarray(coupled_lc)
delta_time = np.asarray(delta_time)
delta_time

array([array([0.       , 0.0204347, 0.0204346, ..., 0.0204347, 0.0204346,
       0.0204347]),
       array([0.       , 0.0408658, 0.0204329, ..., 0.0204347, 0.0204345,
       0.0204348]),
       array([0.       , 0.0204347, 0.0204345, ..., 0.0204346, 0.0204346,
       0.0204347]),
       ...,
       array([0.       , 0.0204343, 0.0204342, ..., 0.0204342, 0.020434 ,
       0.0204343]),
       array([0.       , 0.0204343, 0.0204342, ..., 0.0204342, 0.020434 ,
       0.0204343]),
       array([0.       , 0.0204342, 0.0204342, ..., 0.0204341, 0.0204341,
       0.0204343])], dtype=object)

In [7]:
#cada curva de luz dividirla por la desviación estandar..
for i in range(coupled_lc.shape[0]):
    std_i = coupled_lc[i].std(keepdims=True)
    coupled_lc[i] = coupled_lc[i]/std_i 
coupled_lc.shape

(8054,)

In [8]:
## data augmentation -- como el paper que nos cito...
#flip/mirror cada curva de luz.. (así se duplican los datos..) time: tambn y shift del 0 que sobra..

flip_coupled_lc = []
flip_delta_time = []
for i in range(coupled_lc.shape[0]):    
    flip_coupled_lc.append( coupled_lc_scaled[i][::-1] )
    flip_delta_time.append(  np.hstack([[0], delta_time[i][1:][::-1]]) )
    
flip_coupled_lc = np.asarray(flip_coupled_lc)
flip_delta_time = np.asarray(flip_delta_time)

delta_time = np.concatenate([flip_delta_time,delta_time])
coupled_lc = np.concatenate([flip_coupled_lc,coupled_lc])

del flip_delta_time, flip_coupled_lc
gc.collect()
coupled_lc.shape

(16108,)

### Build representation

In [11]:
max_len = np.max(lens_lc)

## do padding with keras..
X_time = keras.preprocessing.sequence.pad_sequences(delta_time,maxlen=max_len, value=0,dtype='float32',padding='post')
X_lc = keras.preprocessing.sequence.pad_sequences(coupled_lc,maxlen=max_len, value=0,dtype='float32',padding='post')

print("Shape with padding: ", X_lc.shape)
X_lc

Shape with padding:  (16108, 64482)


NameError: name 'X_lc_scaled' is not defined

In [12]:
#add extra dim
X_lc = np.expand_dims(np.squeeze(X_lc),axis=-1)
X_time = np.expand_dims(np.squeeze(X_time),axis=-1)

N_aug, T, _ = X_lc.shape
channels = 2
print("new - Shape: ", X_lc.shape)

new - Shape:  (16108, 64482, 1)


### Define Model

In [13]:
from keras import backend as K
from keras.models import Sequential, Model
from keras.layers import Input, Conv1D, Dense, Flatten, MaxPool1D, Reshape, UpSampling1D, Lambda, AveragePooling1D
from keras.layers import GlobalAveragePooling1D,GlobalMaxPool1D, TimeDistributed, GRU,LSTM, RepeatVector, SimpleRNN
from keras.layers import BatchNormalization, Dropout, ZeroPadding1D, ZeroPadding2D, Cropping1D, Cropping2D, Conv2D, Conv2DTranspose, MaxPool2D,UpSampling2D

from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot

In [14]:
def add_CNN(it, filters, kernel_s, act='relu', dil_r=1, BN = False, **args):
    f1 = Conv1D(filters, kernel_s, activation=act, dilation_rate=dil_r, **args)(it)
    if BN:
        f1 = BatchNormalization()(f1)
    return f1
                                                           
def conv_bloq(it, filters, kernel_s, pool, drop=0, dil_r=1, act='relu', BN=False,double=False, **arg):
    ## **args could be any other conv1d parameter
    f1 = add_CNN(it, filters, kernel_s, act=act, dil_r=dil_r, BN = BN, **args)
    if double:
        f1 = add_CNN(f1, filters, kernel_s, act=act, dil_r=dil_r, BN = BN, **args)
        
    if pool!= 0 and dil_r ==1:
        #f1 = MaxPool1D(pool_size=pool, strides=pool, padding='valid')(f1)
        f1 = AveragePooling1D(pool_size=pool, strides=pool, padding='valid')(f1)
    if drop != 0:
        f1 = Dropout(drop)(f1)
    return f1

def encoder_model_CNN1D(input_dim, channels, L=1, filters=8, kernel_s =10, pool=5, BN=False, drop=0,
                        dil_r=1, flatten =True,time=False,double=False, **args): 
    #parametros estructurales-- args es cuqluier parameter de conv1d
    it = Input(shape=(input_dim,channels))  #fixed length..
    f1 = it
    start_f = filters
    for l in range(L):
        f1 = conv_bloq(f1, filters, kernel_s, pool,conv_pool=conv_pool, drop=drop,dil_r=dil_r,BN=BN,double=double, **args) 
        
        filters = int(filters*2)
        filters = min(128, filters)
        if time:
            filters = min(32, filters)
        if dil_r != 1:
            dil_r = int(dil_r*2)
            
    if flatten:
        f1 = Flatten()(f1)
    else:
        #f1 = GlobalMaxPool1D()(f1)
        f1 = GlobalAveragePooling1D()(f1)
    return Model(inputs=it, outputs=f1)

In [ ]:
EPOCHS = 100
BATCH_s = 256

In [15]:
#Kernel 8 y pool 3
#comenzar con 8 filtros
#en salida deberian haber approx 200-10k o 5k

encoder   = encoder_model_CNN1D(T, 1, L=5, filters=16, pool=3, kernel_s=5, drop=0.0) 
encoder_t = encoder_model_CNN1D(T, 1, L=5, filters=16, pool=3, kernel_s=5, drop=0.0, time=True) #model

encoder.summary()

AttributeError: module 'tensorflow' has no attribute 'get_default_graph'

In [ ]:
## original: C aguirre (T=500) con idea de steps..
encoder  = encoder_model_CNN1D(T, 1, L=2, filters=32, pool=2, kernel_s=40, drop=0.25) 
ncoder_t = encoder # pero la curva son deltas tambn...

encoder.summary()

In [ ]:
## combine two "tower"
it_time = Input(shape=X_time.shape[1:])
it_lc = Input(shape=X_lc.shape[1:])
encoder_time = encoder_t(it_time)
encoder_lc = encoder(it_lc)
concat_encoder = keras.layers.Concatenate()([encoder_time,encoder_lc]) #or multiply?

f1 = Dropout(0.5)(concat_encoder)
f1 = Dense(128, activation='relu')(concat_encoder)
f1 = Dropout(0.5)(f1)
out = Dense(1, activation='sigmoid')(f1)

CNN_all = Model([it_time,it_lc], out)
CNN_all.summary()

### CNN casual+dilation 
> WaveNet

In [ ]:
encoder  = encoder_model_CNN1D(T, 1, L=.., filters=.., kernel_s=.., drop=0.0, flatten=False,dil_r=2, padding='causal' ) 
